In [1]:
# Requirements
%pip install openai -q
%pip install langchain -q
%pip install chromadb -q
%pip install tiktoken -q
%pip install unstructured[local-inference] -q
%pip install pypdf
%pip install gradio -q
%pip install python-dotenv
%pip install tabulate
%pip install huggingface_hub
%pip install InstructorEmbedding
%pip install sentence_transformers


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()

#os.environ["OPENAI_API_KEY"] = ""
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "" 

#OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
#HUGGINGFACEHUB_API_TOKEN=os.getenv("HUGGINGFACEHUB_API_TOKEN")

from langchain.chat_models import ChatOpenAI
#llm = ChatOpenAI(temperature=0,model_name="gpt-4")

llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})

In [13]:
# Data Ingestion
from langchain.document_loaders import DirectoryLoader

pdf_loader = DirectoryLoader('./Reports', glob="**/*.pdf")
txt_loader = DirectoryLoader('./Reports', glob="**/*.txt")
word_loader = DirectoryLoader('./Reports', glob="**/*.docx")

loaders = [pdf_loader, txt_loader, word_loader]
#loaders = [txt_loader, word_loader]
#loaders = [pdf_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")

Total number of documents: 3


In [14]:
# Chunk and Embeddings
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

#embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
vectorstore = Chroma.from_documents(documents, embeddings)

Created a chunk of size 1445, which is longer than the specified 1000
Created a chunk of size 1587, which is longer than the specified 1000
Created a chunk of size 1160, which is longer than the specified 1000


load INSTRUCTOR_Transformer
max_seq_length  512


In [15]:
from langchain.memory import ConversationBufferMemory

# Initialise Langchain - Conversation Retrieval Chain
#qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0), vectorstore.as_retriever(), return_source_documents=True)

memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    #ChatOpenAI(temperature=0),
    llm=HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512}), 
    retriever=vectorstore.as_retriever(), 
    memory=memory,
    #get_chat_history=lambda h : h,
    #return_source_documents=True
    )

#n.b. for ConversationRetrievalChain, history is not a Memory object but an array.  Therefore:
#set memory_key to 'chat_history' (default key names are different between ConversationBufferMemory and ConversationalRetrievalChain)
#adjust get_chat_history to pass through the history from the memory using lambda h : h
#specify 'answer' as the output key when creating the ConversationBufferMemory object as outputs has two keys, answer and source_documents 
#(which throws an error)


In [16]:
# Front end web app
from langchain.memory import ChatMessageHistory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

#Gradio Chatbot, whose value stores the entire history of the conversation as a list of response pairs between the user and the Chatbot
#A single function, respond(), takes in the entire history of the chatbot (chat_history) and appends latest response pair
#It also clears the textbox when it returns

import gradio as gr
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    context = ChatMessageHistory()
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True, output_key='answer')
    
    def respond(user_message, chat_history):
 
        # Get response from QA chain
        response = qa({"question": user_message, "chat_history": chat_history})
        # Append user message and response to chat history
        chat_history.append((user_message, response["answer"]))
        return gr.update(value=""), chat_history
    msg.submit(respond, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
